In [18]:
import ipyvuetify as v
from traitlets import Bool, link, List, observe
from pathlib import Path
from natsort import humansorted
import sepal_ui.scripts.utils as su
from sepal_ui.frontend.styles import *

In [112]:
class ListItem(v.Flex):
    
    # Workaround: It has to be a container: using directly a ListItem
    # will cause that the checkbox is inside the ListItem,
    # So, a item will be selected when the checkbox is checked.
    
    is_selected = Bool().tag(sync=True)
    
    def __init__(self, value, position, *args, **kwargs):
        
        self.value = value
        self.class_='d-flex'
        value = Path(value)
        
        super().__init__(*args, **kwargs)
        
        selected = v.Checkbox(v_model=False)
        
        if position == 0:
            icon = ICON_TYPES['PARENT']["icon"]
            color = ICON_TYPES['PARENT']["color"]
        elif value.suffix in ICON_TYPES.keys():
            icon = ICON_TYPES[value.suffix]["icon"]
            color = ICON_TYPES[value.suffix]["color"]
        else:
            icon = ICON_TYPES["DEFAULT"]["icon"]
            color = ICON_TYPES["DEFAULT"]["color"]      

        self.children = [
            
            v.ListItem(children=[
                v.ListItemAction(
                    children=[
                        v.Icon(color=color, children=[icon])
                    ]),
                v.ListItemContent(
                    children=[v.ListItemTitle(children=[str(value.stem)]),]
                )
            ]),
            selected,
        ]
        link((selected, 'v_model'), (self, 'is_selected'))

In [113]:
class ListItemGroup(v.ListItemGroup):
    
    selected = List([]).tag(sync=None)
    
    def __init__(self, children, *args, **kwargs):
        
        self.v_model=None
        
        super().__init__(*args, **kwargs)
        
        self.children = children
        # Observe childrens
        [item.observe(self.set_selected, 'is_selected') for item in self.children]
    
    def set_selected(self, change):
        self.selected = [(chld.value) for chld in self.children if chld.is_selected]


In [114]:
class FolderSelector(v.Flex):
    
    def __init__(self, *args, **kwargs):
        
        self.folders = None
        
        super().__init__(*args, **kwargs)
        
        self.loading = v.ProgressLinear(
            indeterminate=False,
            background_color="grey darken-3",
            color=COMPONENTS["PROGRESS_BAR"]["color"],
        )
        
        self.get_folder_group(Path('~').expanduser())
        self.children = [
            self.loading,
            self.item_group
        ]
        
    def get_folder_group(self, path):

        self.folders = humansorted([
            str(folder) for folder in Path(path).glob("*/") 
            if (folder.is_dir() and not folder.name.startswith('.'))
        ])
        self.folders = [str(Path(path).parent)] + self.folders
        
        # Each time we have to create a ListItemGroup to have fresh
        # selected items
        self.item_group = ListItemGroup(
            children= [
                ListItem(value=folder, position=i) for i, folder in enumerate(self.folders)
            ]
        )
    
        self.item_group.observe(self.on_folder, 'v_model')
    
    @su.switch('indeterminate', on_widgets=['loading'])
    def on_folder(self, change):

        self.get_folder_group(self.folders[change['new']])
        self.children = [
            self.loading,
            self.item_group
        ]
    


In [115]:
folder_selector = FolderSelector()

In [116]:
folder_selector

FolderSelector(children=[ProgressLinear(background_color='grey darken-3', color='indigo', indeterminate=False)…

In [117]:
folder_selector.item_group.selected

['/home/dguerrero/module_results/planet-order/COL_Antioquia_Alejandria',
 '/home/dguerrero/module_results/planet-order/COL_Antioquia_Amalfi',
 '/home/dguerrero/module_results/planet-order/COL_Antioquia_Cocorna',
 '/home/dguerrero/module_results/planet-order/COL_Arauca',
 '/home/dguerrero/module_results/planet-order/COL_Boyaca_Garagoa',
 '/home/dguerrero/module_results/planet-order/COL_Cundinamarca_Agua_de_Dios',
 '/home/dguerrero/module_results/planet-order/Manual_2021-03-26_13-22-00',
 '/home/dguerrero/module_results/planet-order/Manual_2021-04-21_15-04-34']